In [3]:
# Tensor flow probability
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
?tfd.Bernoulli

Init signature:
tfd.Bernoulli(
    logits=None,
    probs=None,
    dtype=tf.int32,
    validate_args=False,
    allow_nan_stats=True,
    name='Bernoulli',
)
Docstring:     
Bernoulli distribution.

The Bernoulli distribution with `probs` parameter, i.e., the probability of a
`1` outcome (vs a `0` outcome).
Init docstring:
Construct Bernoulli distributions.

Args:
  logits: An N-D `Tensor` representing the log-odds of a `1` event. Each
    entry in the `Tensor` parameterizes an independent Bernoulli
    distribution where the probability of an event is sigmoid(logits). Only
    one of `logits` or `probs` should be passed in.
  probs: An N-D `Tensor` representing the probability of a `1`
    event. Each entry in the `Tensor` parameterizes an independent
    Bernoulli distribution. Only one of `logits` or `probs` should be passed
    in.
  dtype: The type of the event samples. Default: `int32`.
  validate_args: Python `bool`, default `False`. When `True` distribution
    parameters are 

In [12]:
def beta_bernoulli_model():
    model = tfd.JointDistributionNamed({
        "theta": tfd.Beta(concentration0=1, concentration1=1),
        "x_t": lambda theta: tfd.Independent(
            tfd.Bernoulli(probs=theta)
        )
    })
    return model  


In [13]:
model = beta_bernoulli_model()

In [15]:
model.resolve_graph()

(('theta', ()), ('x_t', ('theta',)))

In [23]:
def target_log_prob(theta, data):
    return model.log_prob(theta = theta, x_t = data)

In [24]:
import numpy as np
data = [0,1,0,1]

In [26]:
target_log_prob(0.5, data)

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([-0.6931472, -0.6931472, -0.6931472, -0.6931472], dtype=float32)>

In [31]:
tfb = tfp.bijectors

@tf.function
def make_chain(target_log_prob, n_chains, num_results, num_burin_steps):
    kernel = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=target_log_prob,
        step_size=0.05,
        num_leapfrog_steps=3
    )
    states = tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burin_steps,
        kernel=kernel
    )
    return states


In [32]:
samples = make_chain(target_log_prob, 4, 10000, 3000)

TypeError: in user code:

    <ipython-input-31-a7e499fa0af6>:5 make_chain  *
        kernel = tfp.mcmc.NoUTurnSampler(

    TypeError: __init__() got an unexpected keyword argument 'num_leapfrog_steps'
